# PTT Hate版資料爬蟲

## Install Required Packages

In [2]:
from __future__ import absolute_import
from __future__ import print_function

import os
import re
import sys
import json
import csv
import time
from datetime import datetime
import pandas as pd

import requests
from bs4 import BeautifulSoup
from six import u

全爬下來的話：

1. 文章有「公告」，做停權處置等等，暫不處理
2. 截至2/16共有3990個分頁
3. 目前只爬文章內容，不爬推噓文

## ptt-Crawler 

In [5]:
PTT_URL = 'https://www.ptt.cc'
board = 'Hate'
start_index = 2001
end_index = 3990
all_articles = []

for i in range(end_index - start_index + 1):
    index = start_index + i
    print('Processcing index: ', str(index))
    
    resp = requests.get(url = PTT_URL + '/bbs/' + board + '/index' + str(index) + '.html',
                        cookies = {'over18':'1'})
    
    # status_code=200 is success, so if not success will return error e.g. 404 not found
    if resp.status_code != 200:
        print('invalid url: ', resp.url)
        continue
    
    # parser the html and find articles' div element
    soup = BeautifulSoup(resp.text, 'html.parser')
    divs = soup.find_all('div', 'r-ent')
    
    for div in divs:
        try:
            # for link
            href = div.find('a')['href']
            link = PTT_URL + href
            article_id = re.sub('\.html', '', href.split('/')[-1])

            # print('Processing article: ', article_id)
            # print('link', link) this is for test 
            resp2 = requests.get(url = link, cookies={'over18':'1'})
            
            # status_code=200 is success, so if not success will return error e.g. 404 not found
            if resp2.status_code != 200:
                print('invalid url: ', resp2.url)

            soup2 = BeautifulSoup(resp2.text, 'html.parser')
            main_content2 = soup2.find(id = 'main-content')
            # metaline includes of author, title and datetime
            metas = main_content2.select('div.article-metaline')
            
            author = ''
            title = ''
            date = ''

            if metas:
                author = metas[0].select('span.article-meta-value')[0].string if metas[0].select('span.article-meta-value')[0] else author
                title = metas[1].select('span.article-meta-value')[0].string if metas[1].select('span.article-meta-value')[0] else title
                date = metas[2].select('span.article-meta-value')[0].string if metas[2].select('span.article-meta-value')[0] else date
                
                # 去除ptt中文暱稱
                author = re.sub('\s\(.*?\).', '', author)
                # deal with date format
                date = datetime.strptime(date, '%a %b  %d %H:%M:%S %Y')

                # remove meta nodes
                for m in metas:
                    m.clear()
                for mr in main_content2.select('div.article-metaline-right'):
                    mr.clear()
            
            # remove and keep push nodes
            for p in main_content2.find_all('div', class_='push'):
                p.clear()

            # 移除 '※ 發信站:' (starts with u'\u203b'), '◆ From:' (starts with u'\u25c6'), 空行及多餘空白
            # 保留英數字, 中文及中文標點, 網址, 部分特殊符號
            filtered = [ v for v in main_content2.stripped_strings if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--'] ]
            expr = re.compile(u(r'[^\u4e00-\u9fa5\u3002\uff1b\uff0c\uff1a\u201c\u201d\uff08\uff09\u3001\uff1f\u300a\u300b\s\w:/-_.?~%()]'))
            for i in range(len(filtered)):
                filtered[i] = re.sub(expr, '', filtered[i])

            filtered = [_f for _f in filtered if _f]  # remove empty strings
            filtered = [x for x in filtered if article_id not in x]  # remove last line containing the url of the article
            content = ' '.join(filtered)
            content = re.sub(r'(\s)+', ' ', content)

            data = {
                'url': link,
                'title': title,
                'author': author,
                'date': date,
                'content': content
            }
            all_articles.append(data)
        except:
            print('This article is deleted: ', article_id)
        
    time.sleep(0.05)

Processcing index:  2001
Processcing index:  2002
Processcing index:  2003
Processcing index:  2004
Processcing index:  2005
Processcing index:  2006
Processcing index:  2007
Processcing index:  2008
Processcing index:  2009
Processcing index:  2010
Processcing index:  2011
This article is deleted:  M.1558935595.A.842
Processcing index:  2012
Processcing index:  2013
Processcing index:  2014
Processcing index:  2015
Processcing index:  2016
Processcing index:  2017
Processcing index:  2018
Processcing index:  2019
Processcing index:  2020
Processcing index:  2021
Processcing index:  2022
Processcing index:  2023
Processcing index:  2024
Processcing index:  2025
Processcing index:  2026
Processcing index:  2027
Processcing index:  2028
Processcing index:  2029
Processcing index:  2030
Processcing index:  2031
Processcing index:  2032
Processcing index:  2033
Processcing index:  2034
Processcing index:  2035
Processcing index:  2036
Processcing index:  2037
Processcing index:  2038
Proce

In [6]:
# 上次抓到1085頁 2019/2/17
# store data to pandas and write to csv
all_articles = pd.DataFrame(all_articles)
all_articles.head()
all_articles.to_csv('hate_ptt_3990.csv', index=False)

,author,content,date,title,url
0,angelgirl13 (我是ㄍ胖子),如果我今天12.前能離開 就很棒ㄌ 崩潰QQ 我要來去拿書來讀ㄌ,2019-05-25 21:53:29,[黑特] 讚,https://www.ptt.cc/bbs/Hate/M.1558792412.A.538...
1,sakurainTW (與魔鬼共枕),有些人頗ㄏ 窮醜自己擼 嘻嘻 今天一個40歲娃娃臉的男的一直跟我擠眉弄眼 贏過那種連人在哪都...,2019-05-25 21:59:42,[煩耶] 哈哈哈,https://www.ptt.cc/bbs/Hate/M.1558792784.A.98F...
2,huntersp (史匹),今天晚上一直聽Eason 就這麼快到了要就寢的時間了 時間怎麼這麼快 幹，好浪費啊 馬的 放...,2019-05-25 22:03:35,[超幹],https://www.ptt.cc/bbs/Hate/M.1558793018.A.A50...
3,sakurainTW (與魔鬼共枕),自以為聖人 自以為可以當別人的天使 殊不知這種婦人之仁一時興起的母愛 只會讓那個白癡越來越智...,2019-05-25 22:23:17,[煩耶] 覺得噁心,https://www.ptt.cc/bbs/Hate/M.1558794199.A.965...
4,NBAes (),妳怎不去被車撞? 怎不去死一死 噁心 出來四處釣人 真的死不要臉的厚臉皮婊子 還不知道自己是...,2019-05-25 22:23:58,妳去死一死幹妳娘活在這是屆浪費資源垃圾人幹妳,https://www.ptt.cc/bbs/Hate/M.1558794241.A.A04...


## 以下為 testing code

In [33]:
#### TEST CODE #####
resp = requests.get(url = 'https://www.ptt.cc/bbs/Hate/M.1581756353.A.DC1.html', cookies={'over18':'1'})
# status_code=200 is success, so if not success will return error e.g. 404 not found
if resp.status_code != 200:
    print('invalid url: ', resp.url)

soup = BeautifulSoup(resp.text, 'html.parser')
main_content = soup.find(id = 'main-content')
# metaline includes of author, title and datetime
metas = main_content.select('div.article-metaline')
author = ''
title = ''
date = ''
if metas:
    author = metas[0].select('span.article-meta-value')[0].string if metas[0].select('span.article-meta-value')[0] else author
    title = metas[1].select('span.article-meta-value')[0].string if metas[1].select('span.article-meta-value')[0] else title
    date = metas[2].select('span.article-meta-value')[0].string if metas[2].select('span.article-meta-value')[0] else date#

    # remove meta nodes
#     for meta in metas:
#         meta.extract()
#     for meta in main_content.select('div.article-metaline-right'):
#         meta.extract()

In [34]:
#### TEST CODE #####

from six import u

article_id = 'M.1581756353.A.DC1'

# remove and keep push nodes
# pushes = main_content.find_all('div', class_='push')
# for push in pushes:
#     push.extract()
            
filtered = [ v for v in main_content.stripped_strings if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--'] ]
expr = re.compile(u(r'[^\u4e00-\u9fa5\u3002\uff1b\uff0c\uff1a\u201c\u201d\uff08\uff09\u3001\uff1f\u300a\u300b\s\w:/-_.?~%()]'))
for i in range(len(filtered)):
    filtered[i] = re.sub(expr, '', filtered[i])

filtered = [_f for _f in filtered if _f]  # remove empty strings
print(filtered)
filtered = [x for x in filtered if article_id not in x]  # remove last line containing the url of the article
content = ' '.join(filtered)
print(content)
content = re.sub(r'(\s)+', ' ', content)
print(content)

['作者', 'fengyu118 (煩魚)', '看板', 'Hate', '標題', '[黑特] 夢中夢', '時間', 'Sat Feb 15 16:45:51 2020', '小弟剛剛做了一個可怕的噩夢\n\n夢中小弟被人設計陷害\n\n堅持自己是真的遇到這種爛事\n\n可是沒有人相信我\n\n小弟開始懷疑自己是不是做夢夢到\n但我堅持不是假的\n\n結果醒來以後發現 QQ\n\n', 'https://www.ptt.cc/bbs/Hate/M.1581756353.A.DC1.html', '噓', 'angelgirl13', ':', 'http://i.imgur.com/N30aH6o.jpg', '02/15 16:46', '噓', 'celtics1997', ': 我現在好難過可以打打我嗎', '02/15 16:46', 'fengyu118', ': 13 很氣喔', '02/15 16:46', 'fengyu118', ': 打到你哭著叫媽', '02/15 16:47', '噓', 'lin91223', ': 煩魚我也要    ', '02/15 16:47', 'fengyu118', ': 不要 我要直接把你的毯毯搶走', '02/15 16:47', '推', 'celtics1997', ': 可以看了YAYAYA', '02/15 16:49', '噓', 'lin91223', ': 不行 我要裹著毯毯被打    ', '02/15 16:49', 'lpbrother', ': 我都夢到樓下', '02/15 16:51', 'celtics1997', ': 大家快去看Netflix YAYAYA', '02/15 16:51', '噓', 'celtics1997', ': Outlander上了YAYAYA 馬眼YA透ㄌ...', '02/15 16:55', 'celtics1997', ': 好爽喔喔喔喔喔喔喔', '02/15 16:55', 'celtics1997', ': 喔喔喔喔喔喔喔欸欸', '02/15 16:55', 'celtics1997', ': AAAAAAA', '02/15 16:55']
作者 fengyu118 (煩魚) 看板 Hate 標題 [黑特] 夢中夢 時間 Sa